In [52]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import pandas as pd
import re

import numpy as np

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.metrics import roc_auc_score

In [2]:
df_main = pd.read_csv('./driver_policy.csv')
vehicles_data = pd.read_csv('./vehicles.csv')

In [3]:
vehicles_data

,Unnamed: 0,policy_id,car_no,ownership_type,color,age,make_model
0,6,policy_74571,1,leased,other,4.0,BMW : R1200CL
1,7,policy_74571,2,owned,blue,3.0,ACURA : TL
2,15,policy_1998,1,owned,gray,7.0,BMW : 750LI
3,29,policy_28085,1,owned,gray,4.0,MERCEDES-BENZ : G55 AMG
4,34,policy_64282,1,owned,black,4.0,BMW : 550I GT
...,...,...,...,...,...,...,...
169232,345054,policy_34204,4,owned,gray,6.0,TOYOTA : RAV4
169233,345055,policy_34204,5,loaned,black,8.0,GMC : SIERRA 1500
169234,345060,policy_621,1,owned,other,6.0,DODGE : CHARGER
169235,345061,policy_621,2,owned,black,3.0,GMC : SAVANA 4500


In [4]:
vehicles_data = vehicles_data.drop(['Unnamed: 0', 'ownership_type', 'color', 'make_model'], axis='columns')

In [5]:
vehicles_data

,policy_id,car_no,age
0,policy_74571,1,4.0
1,policy_74571,2,3.0
2,policy_1998,1,7.0
3,policy_28085,1,4.0
4,policy_64282,1,4.0
...,...,...,...
169232,policy_34204,4,6.0
169233,policy_34204,5,8.0
169234,policy_621,1,6.0
169235,policy_621,2,3.0


In [6]:
new = pd.DataFrame(vehicles_data.groupby('policy_id')['car_no'].max())

In [7]:
new = new.reset_index()

In [8]:
vehicles_data[vehicles_data['policy_id']=='policy_99996']['age'].sum()

15.0

In [9]:
list_sum_age = []
for i in new['policy_id']:
    
    list_sum_age.append(vehicles_data[vehicles_data['policy_id']==i]['age'].sum())
    

In [10]:
len(list_sum_age)

49162

In [11]:
new['sum_age'] = list_sum_age

In [12]:
new

,policy_id,car_no,sum_age
0,policy_100,6,44.0
1,policy_1000,2,11.0
2,policy_10001,2,10.0
3,policy_10002,5,36.0
4,policy_10004,2,10.0
...,...,...,...
49157,policy_99994,4,30.0
49158,policy_99996,4,15.0
49159,policy_99997,4,29.0
49160,policy_99998,1,3.0


In [13]:
df_merged = pd.merge(df_main, new)

In [14]:
df_merged = df_merged.drop(['Unnamed: 0', 'X', 'discount', 'Home_policy_ind', 'zip', 'state_id', 'county_name'
                           , 'primary_parking'], axis='columns')

In [15]:
df_merged = df_merged.drop(['Prior_carrier_grp', 'Cov_package_type'], axis = 'columns')

In [16]:
df_merged = df_merged.drop(['Quote_dt', 'Agent_cd', 'CAT_zone'], axis = 'columns')

In [17]:
df_merged

,quoted_amt,credit_score,policy_id,number_drivers,num_loaned_veh,num_owned_veh,num_leased_veh,total_number_veh,convert_ind,split,driver_age,n_safty_rating,car_no,sum_age
0,"$5,153",613.0,policy_87209,2,1,2,0,3,0.0,Train,99,148.0,4,21.0
1,"$9,870",576.0,policy_89288,1,1,1,2,4,NaN,Test,56,62.0,4,11.0
2,"$3,090",631.0,policy_91413,2,1,2,1,4,0.0,Train,98,140.0,4,28.0
3,"$2,860",573.0,policy_23460,1,2,3,0,5,NaN,Test,30,90.0,2,11.0
4,"$14,917",602.0,policy_71845,2,0,1,2,3,0.0,Train,113,163.0,4,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49157,"$4,042",590.0,policy_28284,1,2,2,0,4,0.0,Train,52,90.0,4,20.0
49158,"$5,743",688.0,policy_67016,2,2,2,0,4,0.0,Train,138,147.0,4,25.0
49159,"$2,230",637.0,policy_30163,1,1,3,1,5,0.0,Train,19,39.0,2,14.0
49160,"$8,428",581.0,policy_63982,2,2,2,0,4,1.0,Train,106,156.0,5,24.0


In [18]:
df_merged.isna().sum()

quoted_amt            112
credit_score          300
policy_id               0
number_drivers          0
num_loaned_veh          0
num_owned_veh           0
num_leased_veh          0
total_number_veh        0
convert_ind         12291
split                   0
driver_age              0
n_safty_rating         77
car_no                  0
sum_age                 0
dtype: int64

In [19]:
df_train = df_merged[df_merged['split']=='Train']

In [20]:
df_train

,quoted_amt,credit_score,policy_id,number_drivers,num_loaned_veh,num_owned_veh,num_leased_veh,total_number_veh,convert_ind,split,driver_age,n_safty_rating,car_no,sum_age
0,"$5,153",613.0,policy_87209,2,1,2,0,3,0.0,Train,99,148.0,4,21.0
2,"$3,090",631.0,policy_91413,2,1,2,1,4,0.0,Train,98,140.0,4,28.0
4,"$14,917",602.0,policy_71845,2,0,1,2,3,0.0,Train,113,163.0,4,18.0
5,"$4,620",704.0,policy_29027,1,2,2,0,4,0.0,Train,18,66.0,5,30.0
6,"$11,470",611.0,policy_75562,4,2,2,2,6,0.0,Train,93,297.0,3,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49157,"$4,042",590.0,policy_28284,1,2,2,0,4,0.0,Train,52,90.0,4,20.0
49158,"$5,743",688.0,policy_67016,2,2,2,0,4,0.0,Train,138,147.0,4,25.0
49159,"$2,230",637.0,policy_30163,1,1,3,1,5,0.0,Train,19,39.0,2,14.0
49160,"$8,428",581.0,policy_63982,2,2,2,0,4,1.0,Train,106,156.0,5,24.0


In [21]:
df_test = df_merged[df_merged['split']=='Test']

In [22]:
df_test

,quoted_amt,credit_score,policy_id,number_drivers,num_loaned_veh,num_owned_veh,num_leased_veh,total_number_veh,convert_ind,split,driver_age,n_safty_rating,car_no,sum_age
1,"$9,870",576.0,policy_89288,1,1,1,2,4,NaN,Test,56,62.0,4,11.0
3,"$2,860",573.0,policy_23460,1,2,3,0,5,NaN,Test,30,90.0,2,11.0
7,"$2,980",679.0,policy_43809,4,0,2,0,2,NaN,Test,91,238.0,2,6.0
8,"$1,945",773.0,policy_4590,3,1,3,2,6,NaN,Test,144,194.0,4,9.0
9,"$5,829",680.0,policy_65525,1,2,3,0,5,NaN,Test,55,72.0,5,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49137,"$2,922",727.0,policy_8331,1,3,3,0,6,NaN,Test,43,28.0,4,17.0
49144,"$5,437",658.0,policy_4090,5,1,2,0,3,NaN,Test,133,318.0,4,19.0
49146,"$3,287",586.0,policy_39956,1,2,2,0,4,NaN,Test,18,51.0,1,6.0
49150,"$3,567",693.0,policy_68766,3,0,2,1,3,NaN,Test,87,183.0,4,27.0


In [23]:
df_train.isna().sum()

quoted_amt           87
credit_score        224
policy_id             0
number_drivers        0
num_loaned_veh        0
num_owned_veh         0
num_leased_veh        0
total_number_veh      0
convert_ind           0
split                 0
driver_age            0
n_safty_rating       62
car_no                0
sum_age               0
dtype: int64

In [24]:
df_train = df_train.dropna(axis=0)

In [25]:
df_train = df_train.reset_index()

In [26]:
df_train['quoted_amt']

0         $5,153
1         $3,090
2        $14,917
3         $4,620
4        $11,470
          ...   
36494     $4,042
36495     $5,743
36496     $2,230
36497     $8,428
36498     $1,220
Name: quoted_amt, Length: 36499, dtype: object

In [27]:
list_quoted_price = []
for i in range(len(df_train)):
    list_quoted_price.append(int(re.sub(r'[^0-9]', '', df_train['quoted_amt'][i])))

In [28]:
df_train = df_train.drop(['quoted_amt'], axis = 'columns')
df_train['quoted_amt'] = list_quoted_price


In [29]:
df_train = df_train.drop(['index', 'policy_id', 'split'], axis = 'columns')

In [30]:
df_train

,credit_score,number_drivers,num_loaned_veh,num_owned_veh,num_leased_veh,total_number_veh,convert_ind,driver_age,n_safty_rating,car_no,sum_age,quoted_amt
0,613.0,2,1,2,0,3,0.0,99,148.0,4,21.0,5153
1,631.0,2,1,2,1,4,0.0,98,140.0,4,28.0,3090
2,602.0,2,0,1,2,3,0.0,113,163.0,4,18.0,14917
3,704.0,1,2,2,0,4,0.0,18,66.0,5,30.0,4620
4,611.0,4,2,2,2,6,0.0,93,297.0,3,21.0,11470
...,...,...,...,...,...,...,...,...,...,...,...,...
36494,590.0,1,2,2,0,4,0.0,52,90.0,4,20.0,4042
36495,688.0,2,2,2,0,4,0.0,138,147.0,4,25.0,5743
36496,637.0,1,1,3,1,5,0.0,19,39.0,2,14.0,2230
36497,581.0,2,2,2,0,4,1.0,106,156.0,5,24.0,8428


In [31]:
x_train = df_train.drop(['convert_ind'], axis='columns')
y_train = df_train['convert_ind']

In [32]:
x = np.asarray(x_train)
y = np.asarray(y_train)


x_train_train, x_train_test, y_train_train, y_train_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [33]:
model = XGBClassifier(random_state=42)


In [34]:
x_train_train

array([[6.550e+02, 2.000e+00, 2.000e+00, ..., 6.000e+00, 3.400e+01,
        4.616e+03],
       [5.770e+02, 1.000e+00, 2.000e+00, ..., 3.000e+00, 2.100e+01,
        7.447e+03],
       [6.040e+02, 2.000e+00, 1.000e+00, ..., 2.000e+00, 1.500e+01,
        5.407e+03],
       ...,
       [5.500e+02, 3.000e+00, 2.000e+00, ..., 3.000e+00, 1.400e+01,
        7.769e+03],
       [5.880e+02, 1.000e+00, 2.000e+00, ..., 2.000e+00, 1.100e+01,
        6.758e+03],
       [4.790e+02, 5.000e+00, 1.000e+00, ..., 2.000e+00, 9.000e+00,
        7.544e+03]])

In [35]:
y_train_train

array([0., 0., 0., ..., 0., 0., 0.])

In [36]:
model.fit(x_train_train, y_train_train)

XGBClassifier(random_state=42)

In [37]:
from sklearn.metrics import accuracy_score

In [38]:
y_pred = model.predict(x_train_test)

In [39]:
accuracy_score(y_pred, y_train_test)

0.8887671232876713

In [70]:
import catboost
from catboost import CatBoostClassifier

clf_catboost = CatBoostClassifier(iterations=100, random_state=42)
clf_catboost.fit(x_train_train, y_train_train, eval_set=(x_train_test, y_train_test))

Learning rate set to 0.198543
0:	learn: 0.5854141	test: 0.5854268	best: 0.5854268 (0)	total: 2.88ms	remaining: 285ms
1:	learn: 0.5141890	test: 0.5140569	best: 0.5140569 (1)	total: 5.15ms	remaining: 252ms
2:	learn: 0.4653446	test: 0.4651484	best: 0.4651484 (2)	total: 7.64ms	remaining: 247ms
3:	learn: 0.4305501	test: 0.4303389	best: 0.4303389 (3)	total: 9.88ms	remaining: 237ms
4:	learn: 0.4057409	test: 0.4056365	best: 0.4056365 (4)	total: 12.6ms	remaining: 239ms
5:	learn: 0.3881740	test: 0.3882024	best: 0.3882024 (5)	total: 16.1ms	remaining: 252ms
6:	learn: 0.3754685	test: 0.3754999	best: 0.3754999 (6)	total: 18.7ms	remaining: 248ms
7:	learn: 0.3659408	test: 0.3661273	best: 0.3661273 (7)	total: 21ms	remaining: 242ms
8:	learn: 0.3594617	test: 0.3597418	best: 0.3597418 (8)	total: 23.6ms	remaining: 239ms
9:	learn: 0.3542958	test: 0.3547044	best: 0.3547044 (9)	total: 26.7ms	remaining: 240ms
10:	learn: 0.3506180	test: 0.3511605	best: 0.3511605 (10)	total: 29.3ms	remaining: 237ms
11:	learn: 0.

In [71]:
cb_auc = roc_auc_score(y_train_test, clf_catboost.predict_proba(x_train_test)[:,1])
print("AUC score of CatBoost: {:.3f}".format(cb_auc))

AUC score of CatBoost: 0.642


In [ ]:
### how to deal with categorical data?
### hyperparameter setting?